<a href="https://colab.research.google.com/github/francotestori/aauba_02/blob/master/armado_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico 2

## Armado del dataset

In [6]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
!mkdir speechcommands
!tar -xf speech_commands_v0.01.tar.gz -C /content/speechcommands

--2020-06-29 21:53:59--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.200.128, 2607:f8b0:4001:c16::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.200.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz.1’

speech_commands_v0. 100%[===================>]   1.39G   216MB/s    in 8.8s    

2020-06-29 21:54:08 (161 MB/s) - ‘speech_commands_v0.01.tar.gz.1’ saved [1489096277/1489096277]

mkdir: cannot create directory ‘speechcommands’: File exists


In [7]:
# Import de Librerías
import librosa
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import Audio
from librosa.display import specshow

In [8]:
# Busco y guardo en un mapa todos los audio files correspondientes a los digit_keys
digit_command_keys = [
  'one',
  'two',
  'three',
  'four',
  'five',
  'six',
  'seven',
  'eight',
  'nine'
]

audio_files = {}

for key in digit_command_keys:
  directory = f'speechcommands/{key}/*.wav'
  audio_files[key] = glob.glob(directory)

In [9]:
# Utilizamos la función propuesta en el TP 
# para hacer la extracción de los datos 
# correspondientes a cada audio file

def calculate_features(
    filename,
    n_mfcc=12,
    delta=True,
    deltadelta=True,
    energy=True, 
    summary_fn = [np.mean, np.std], 
    summary_names=['mean','std']):
  #Abro el archivo:
  x, sr = librosa.core.load(filename,sr=None)
  
  #Calculo MFCCs
  features = librosa.feature.mfcc(x,sr=sr,n_mfcc=n_mfcc)
  feat_names = ['mfcc_{}'.format(i) for i in range(n_mfcc)]
  #Calculo energia:
  if energy:
    energy = librosa.feature.rmse(x)
    features = np.concatenate([features,energy])
    feat_names = feat_names + ['energy']
  #Aplico media y desvio estandar por defecto
  summary_features = np.concatenate([fn(features,axis=1) for fn in summary_fn])
  feat_names = ['{}_{}'.format(name_i,summ_i) for summ_i in summary_names for name_i in feat_names]
  
  #Lo mismo con los delta
  if delta:
    deltafeatures = np.diff(features)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltafeatures,axis=1) for fn in summary_fn])])
    d_names = ['d{}'.format(name) for name in feat_names]
  else:
    d_names = []

  #Y con los delta de segundo orden
  if deltadelta:
    deltadeltafeatures = np.diff(features,n=2)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltadeltafeatures,axis=1) for fn in summary_fn])]) 
    dd_names = ['dd{}'.format(name) for name in feat_names]
  else:
    dd_names = []

  feat_names = feat_names + d_names + dd_names

  return summary_features, feat_names  

In [10]:
audio_df = []

for key in audio_files:
  for audio in audio_files[key]:
    feat, names = calculate_features(audio)
    audio_row = dict(zip(names, feat))
    audio_row['digit'] = key

    audio_df.append(audio_row)
  
audio_df = pd.DataFrame(audio_df)

In [11]:
audio_df

,mfcc_0_mean,mfcc_1_mean,mfcc_2_mean,mfcc_3_mean,mfcc_4_mean,mfcc_5_mean,mfcc_6_mean,mfcc_7_mean,mfcc_8_mean,mfcc_9_mean,mfcc_10_mean,mfcc_11_mean,energy_mean,mfcc_0_std,mfcc_1_std,mfcc_2_std,mfcc_3_std,mfcc_4_std,mfcc_5_std,mfcc_6_std,mfcc_7_std,mfcc_8_std,mfcc_9_std,mfcc_10_std,mfcc_11_std,energy_std,dmfcc_0_mean,dmfcc_1_mean,dmfcc_2_mean,dmfcc_3_mean,dmfcc_4_mean,dmfcc_5_mean,dmfcc_6_mean,dmfcc_7_mean,dmfcc_8_mean,dmfcc_9_mean,dmfcc_10_mean,dmfcc_11_mean,denergy_mean,dmfcc_0_std,dmfcc_1_std,dmfcc_2_std,dmfcc_3_std,dmfcc_4_std,dmfcc_5_std,dmfcc_6_std,dmfcc_7_std,dmfcc_8_std,dmfcc_9_std,dmfcc_10_std,dmfcc_11_std,denergy_std,ddmfcc_0_mean,ddmfcc_1_mean,ddmfcc_2_mean,ddmfcc_3_mean,ddmfcc_4_mean,ddmfcc_5_mean,ddmfcc_6_mean,ddmfcc_7_mean,ddmfcc_8_mean,ddmfcc_9_mean,ddmfcc_10_mean,ddmfcc_11_mean,ddenergy_mean,ddmfcc_0_std,ddmfcc_1_std,ddmfcc_2_std,ddmfcc_3_std,ddmfcc_4_std,ddmfcc_5_std,ddmfcc_6_std,ddmfcc_7_std,ddmfcc_8_std,ddmfcc_9_std,ddmfcc_10_std,ddmfcc_11_std,ddenergy_std,digit
0,-391.940198,43.564312,-5.199797,19.023052,-5.923703,9.225527,1.367611,10.620706,-5.997478,9.574374,-8.517503,10.713215,0.013336,78.119624,21.489048,21.252028,19.643546,14.829658,15.489101,9.616451,6.609624,7.836877,5.359365,10.980826,7.755112,0.015954,0.229550,0.046851,-0.051693,-0.238630,0.036575,0.136785,0.117911,0.122906,-0.009580,0.181499,-0.016226,-0.070427,1.220936e-05,24.382868,13.515582,12.780422,10.146198,8.486261,10.490636,8.076773,4.542802,6.366297,4.631907,4.971850,5.707085,0.004850,-0.194531,-0.265728,-0.384428,-0.265502,0.038387,-0.158966,-0.105365,0.032907,-0.131039,-0.226609,0.103187,0.063885,-3.277747e-06,17.463151,14.918777,10.386825,8.557335,9.999005,10.961163,8.267898,5.803701,7.207028,5.675919,6.069015,6.737806,0.002849,one
1,-329.744111,106.007465,0.015588,28.149881,2.190863,-4.945757,1.565915,-8.536740,2.165148,8.047676,-19.064751,-7.619006,0.092005,175.901040,44.092582,31.677909,22.456294,22.675448,15.638339,13.126369,9.219843,14.300418,14.638729,9.407380,6.894813,0.119845,-0.798732,0.802872,1.066209,-0.537549,0.758310,0.438305,-0.711957,0.725688,0.614179,-0.304611,0.559497,0.396167,-2.294230e-05,51.837799,24.647756,15.351426,12.319363,10.740749,9.883396,7.480433,6.273290,7.748168,6.242858,5.974833,4.738549,0.036590,-0.331860,-0.131764,-0.006622,-0.009887,0.247171,-0.017458,-0.246164,0.117665,0.123099,-0.151847,-0.195053,0.061576,2.197246e-06,39.161585,27.535114,16.983100,12.914879,10.591369,14.108059,8.736058,7.509045,9.058153,7.925219,6.822946,5.547106,0.023304,one
2,-556.857144,88.351723,3.011961,23.468485,-2.340489,7.236673,-13.442327,-11.912243,-7.937271,-21.368699,-22.896851,-2.912486,0.012592,138.061442,57.576256,42.139807,25.859318,29.165731,11.731135,19.814894,19.595829,11.217589,15.340973,9.926126,10.915038,0.012814,0.381976,-0.810892,-1.232607,-0.473179,-0.295119,-0.181584,-0.184720,-0.077257,0.018737,0.395811,0.304715,0.051346,-1.740563e-04,44.845550,21.267395,20.398653,14.342822,15.219662,8.851301,12.340496,7.993034,7.922248,6.434260,5.440362,7.170498,0.002833,-3.303314,-2.825191,-1.658707,-0.839723,-0.068700,0.187568,0.446114,0.537938,0.766495,0.411539,0.471737,0.228541,-1.550963e-04,36.236521,16.798214,15.098885,12.765607,12.923897,8.265046,12.984232,7.575817,9.157174,6.887394,7.364729,8.308755,0.001042,one
3,-471.962503,133.777789,5.335323,14.638465,-7.829380,-18.061956,-12.620131,-6.264448,-2.393828,-6.469114,-11.184625,-6.081257,0.026417,152.035305,76.649915,42.810801,13.994420,28.381246,16.469944,14.429256,7.525183,10.258039,9.106793,10.705910,8.572212,0.031627,-0.962776,-0.937843,-0.038363,0.747919,1.121642,0.897474,0.272668,-0.073555,0.011579,0.064105,-0.023925,-0.094680,-2.280228e-06,39.068380,27.364014,21.438948,9.529633,11.849828,9.327385,8.609497,6.046697,4.311212,7.307763,5.781091,4.197118,0.009728,-0.181522,-0.031313,0.199763,0.133602,0.007044,0.062999,0.278290,0.279201,0.136430,0.174469,0.264966,0.231041,-5.846659e-07,23.064092,24.513044,19.743626,11.392067,7.936353,8.593047,9.135986,6.785102,5.3308

In [12]:
# Guardamos nuestro dataframe en un csv
audio_df.to_csv(r'/content/audio_digits.csv')